In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']= spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-3.2.3/spark-3.2.3-bin-hadoop2.7.tgz
!tar xf spark-3.2.3-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.2.3-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 2s (207 kB/s)
Reading package lists... Done


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.4.0.jar

--2023-03-28 03:05:52--  https://jdbc.postgresql.org/download/postgresql-42.4.0.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042588 (1018K) [application/java-archive]
Saving to: ‘postgresql-42.4.0.jar.1’

postgresql-42.4.0.j 100%[===================>]   1018K  6.42MB/s    in 0.2s    

2023-03-28 03:05:53 (6.42 MB/s) - ‘postgresql-42.4.0.jar.1’ saved [1042588/1042588]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.4.0.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   50423057|R135Q3VZ4DQN5N|B00JWXFDMG|     657335467|Everbling Purple ...|         Jewelry|          5|            0|          0|   N|                Y|           Beauties!|so beautiful even...| 2015-08-31|
|         US|   11262325|R2N0QQ6R4T7YRY|B00W5T1H9W|      26030170|925 Sterling Silv...|         Jewelry|          5|    

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [6]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   45657423|             1|
|   47663761|             1|
|   11931196|             1|
|   44620216|             1|
|    1167081|             1|
|   17682867|             1|
|   10782454|             1|
|   51881951|             1|
|    1537684|             1|
|   17133428|             5|
|     661760|             1|
|    9867391|             9|
|   31268594|             1|
|   48581590|             1|
|   32466409|             3|
|   38490515|             1|
|   41318181|             4|
|   18800155|             6|
|   46306239|             1|
|   28409312|             1|
+-----------+--------------+
only showing top 20 rows



In [7]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(['product_id', 'product_title']).drop_duplicates(['product_id'])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|109485901X|Lovely Cute Balle...|
|1429128054|Ring Praise to th...|
|1438809484|Twilight New Moon...|
|2047037255|World Pride Handl...|
|3293015344|Blessed by Pope B...|
|3979004996|Holy Land Necklac...|
|6040383305|Elastic Raw Balti...|
|6040384441|Authentic Unpolis...|
|6040385596|Elastic Raw Balti...|
|6152004780|The Vampire Diari...|
|6167521514|Antique Silver Un...|
|7106116521|Milliongadgets(TM...|
|7211045639|WinWin Korean Fas...|
|7482782788|Unisex Genuine Co...|
|7482784861|Saint St Michael ...|
|7842955867|Blessed By Pope B...|
|7842955883|Saint Benedict Bl...|
|8197207615|Milliongadgets(TM...|
|874224076X|Our Lady of Good ...|
|8742243106|St Damiano Cross ...|
+----------+--------------------+
only showing top 20 rows



In [8]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(['review_id', 'customer_id', 'product_id', 'product_parent', to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R135Q3VZ4DQN5N|   50423057|B00JWXFDMG|     657335467| 2015-08-31|
|R2N0QQ6R4T7YRY|   11262325|B00W5T1H9W|      26030170| 2015-08-31|
|R3N5JE5Y4T6W5M|   27541121|B00M2L6KFY|     697845240| 2015-08-31|
|R2I150CX5IVY9Q|    5350721|B0006SW2WU|     569859289| 2015-08-31|
|R1RM9ICOOA9MQ3|   24484424|B009YPDW70|     332947422| 2015-08-31|
|R2J2KMDL10UMSH|     457767|B00QGA8XZM|     424804582| 2015-08-31|
|R3R9ZUFA4TB4FQ|   31009358|B00L0N0E8E|     589441783| 2015-08-31|
|R3UQ8VAQN7R6WL|    7034628|B00DMWQKWA|     517428445| 2015-08-31|
|R1FXZ69C01JNQM|   34608636|B00THLHAGA|     320313750| 2015-08-31|
| RY36LB4OW0FFS|   44972687|B00UXSEOHY|     589365488| 2015-08-31|
| R6PKS54PC6ZY2|   16684508|B00XGRYGVS|     217045013| 2015-08-31|
|R1LYSAH0IECVWB|   23493178|B00AJ2VU8A|     972853757| 2015-08

In [9]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R135Q3VZ4DQN5N|          5|            0|          0|   N|                Y|
|R2N0QQ6R4T7YRY|          5|            0|          0|   N|                N|
|R3N5JE5Y4T6W5M|          5|            0|          0|   N|                Y|
|R2I150CX5IVY9Q|          5|            0|          0|   N|                Y|
|R1RM9ICOOA9MQ3|          5|            0|          0|   N|                Y|
|R2J2KMDL10UMSH|          5|            0|          0|   N|                Y|
|R3R9ZUFA4TB4FQ|          5|            0|          0|   N|                Y|
|R3UQ8VAQN7R6WL|          5|            0|          0|   N|                Y|
|R1FXZ69C01JNQM|          5|            0|          0|   N|                Y|
| RY36LB4OW0FFS|          5|            0|          0|   N|     

In [10]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://challenge17-db.c990ypk4kttu.us-east-2.rds.amazonaws.com/challenge"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [11]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [12]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [13]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [15]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)